In [1]:
import numpy as np

In [38]:
class NeuralNetwork_4layers():
    
    def __init__(self, num_hidden_neurons = 32, learning_rate = 0.01, epochs = 3000):
        # Layer 0 --> input, Layer 1 --> hidden, Layer 2 --> output
        np.random.seed(0)
        self.w_layer01 = np.random.rand(16, num_hidden_neurons)
        self.w_layer12 = np.random.rand(num_hidden_neurons, num_hidden_neurons)
        self.w_layer23 = np.random.rand(num_hidden_neurons, 1)
        self.num_hidden_neurons = num_hidden_neurons
        self.learning_rate = learning_rate
        self.epochs = epochs
        
    def non_linearity(self, value, derivative = False):
        if derivative == True:
            return value*(1. - value)
        value = np.array(value, dtype = np.float128)
        value = np.clip( value, -500, 500 )
        return 1./(1. + np.exp(-value))
        
    def train(self, input_array, output_array):
        for i in range(self.epochs+1):
            layer0 = input_array
            layer1_local_field = input_array.dot(self.w_layer01)
            layer1_output = self.non_linearity(layer1_local_field)
        
            layer2_local_field = layer1_output.dot(self.w_layer12)
            layer2_output = self.non_linearity(layer2_local_field)
            
            layer3_local_field = layer2_output.dot(self.w_layer23)
            layer3_output = self.non_linearity(layer3_local_field)
        
            layer3_error = layer3_output - output_array
            layer3_delta = layer3_error * self.non_linearity(layer3_error, derivative = True)
        
            layer2_error = layer3_delta.dot(self.w_layer23.T)
            layer2_delta = layer2_error * self.non_linearity(layer2_error, derivative = True)
            
            layer1_error = layer2_delta.dot(self.w_layer12.T)
            layer1_delta = layer1_error * self.non_linearity(layer1_error, derivative = True)
        
            self.w_layer23 -= self.learning_rate * layer2_output.T.dot(layer3_delta)
            self.w_layer12 -= self.learning_rate * layer1_output.T.dot(layer2_delta)
            self.w_layer01 -= self.learning_rate * layer0.T.dot(layer1_delta)
            
            if i%100 == 0:
                print 'iteration: ', i
                print 'error: ', np.mean(np.abs(layer3_error))
                
    def save_learned_weights(self):
        np.savetxt('w_layer23.txt', self.w_layer23 , delimiter=' ')
        np.savetxt('w_layer12.txt', self.w_layer12 , delimiter=' ')
        np.savetxt('w_layer01.txt', self.w_layer01 , delimiter=' ')
        
    def load_saved_weights(self):
        self.w_layer23 = np.loadtxt('data/16bits/weights_16bit_32hidden/w_layer23.txt')
        self.w_layer12 = np.loadtxt('data/16bits/weights_16bit_32hidden/w_layer12.txt')
        self.w_layer01 = np.loadtxt('data/16bits/weights_16bit_32hidden/w_layer01.txt')
                
    def predict(self, input_array, desired_array= None):
        layer0 = input_array
        layer1_local_field = input_array.dot(self.w_layer01)
        layer1_output = self.non_linearity(layer1_local_field)
        
        layer2_local_field = layer1_output.dot(self.w_layer12)
        layer2_output = self.non_linearity(layer2_local_field)
            
        layer3_local_field = layer2_output.dot(self.w_layer23)
        layer3_output = self.non_linearity(layer3_local_field)
        
        if desired_array != None:
            layer3_error = layer3_output - desired_array
            print 'accuracy is ', (1.- np.mean(np.abs(layer3_error))) * 100, '%'
        else:
            prime_class = int(layer3_output > 0.5)
            print 'Predicted class: ', prime_class
            print 'Confidence in percentage: ', (
                np.abs(1 - np.round(layer3_output, 4))*100), '%'

network = NeuralNetwork_4layers()

In [39]:
train_set_data = np.loadtxt('data/16bits/train_set_data_16bit.txt')
train_set_labels = np.loadtxt('data/16bits/train_set_labels_16bit.txt').reshape(-1,1)
test_set_data = np.loadtxt('data/16bits/test_set_data_16bit.txt')
test_set_labels = np.loadtxt('data/16bits/test_set_labels_16bit.txt').reshape(-1,1)
print train_set_data.shape
print train_set_labels.shape
print test_set_data.shape
print test_set_labels.shape

(60000, 16)
(60000, 1)
(5535, 16)
(5535, 1)


In [40]:
mini_batch_size = 24
train_data_batch = np.vsplit(train_set_data, mini_batch_size)
train_labels_batch = np.vsplit(train_set_labels, mini_batch_size)

In [ ]:
for i in range(mini_batch_size-23):
    print 'running mini batch number: ', i+1
    network.train(train_data_batch[i], train_labels_batch[i])

In [41]:
#network.save_learned_weights()
network.predict(test_set_data, test_set_labels)

accuracy is  10.2439053938 %


/home/nareshguru77/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:70: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [42]:
network.load_saved_weights()
to_predict = np.array([57]).reshape(-1,1)
to_predict = to_predict >> np.arange(16)[::-1] & 1
print to_predict
network.predict(to_predict)

[[0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1]]
Predicted class:  0
Confidence in percentage:  [ 100.0] %
